<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/stock_predict_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
# from google.colab import drive
# drive.mount('/content/drive')
from sklearn.preprocessing import StandardScaler
import numpy as np
import torch
from torch import nn

In [ ]:
# 데이터 불러오기
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/Microsoft_Stock.csv')

In [ ]:
# 날짜데이터 누락
data.drop('Date', axis=1, inplace=True)

In [ ]:
data

In [ ]:
n_data = data.to_numpy()
n_data.shape

In [ ]:
# # 데이터 표준화
# # 지수적으로 증가하는 주가데이터 특성상 전체를 한번에 표준화할 경우 초기데이터의 와 후기데이터의 scale차 발생.
# scaler = StandardScaler()

# std_data = scaler.fit_transform(data) # numpy 배열로 반환함.

In [ ]:
# 1부터 10 사이의 정수 중에서 중복 없이 랜덤하게 5개 추출
s = np.random.choice(range(0, 1511-81), size=30, replace=False)

print(s)

In [ ]:
def data_gen(Size:int):

    s = np.random.choice(range(0, 1511-81), size=Size, replace=False)
    s = np.array(s)

    short_term_data = []
    short_term_label = []

    for i in s:
        s_data = n_data[i:i+30, :]
        short_term_data.append(s_data)

        label = (n_data[i+51,2] - n_data[i+31,2]) / n_data[i+31,2] * 100

        if label >= 10:
            short_term_label.append(3)

        elif 0 <= label < 10:
            short_term_label.append(2)

        elif -10 < label < 0:
            short_term_label.append(1)

        elif label <= -10:
            short_term_label.append(0)

    short_term_data = np.array(short_term_data)
    short_term_label = np.array(short_term_label)

    return short_term_data, short_term_label

short_term_data, short_term_label = data_gen(60)

In [ ]:
short_term_data.shape

In [ ]:
short_term_label.shape

In [ ]:
class RNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.rnn = nn.RNN(5,30,3,batch_first = True)
        self.linear = nn.Linear(30,4)

    def forward(self, x):
        _, x = self.rnn(x)
        x = x[-1,:,:]
        # x = x.squeeze(0)
        x = self.linear(x)
        return x

model = RNN()

In [ ]:
model.state_dict

In [ ]:
from torchsummary import summary
from contextlib import redirect_stdout

model = RNN()

# 요약된 모델 아키텍처를 직접 파일에 리디렉션합니다.
with open('/content/drive/MyDrive/Colab Notebooks/graph/model_summary.txt', 'w') as f:
    with redirect_stdout(f):
        summary(model, (30, 5))

print("모델 아키텍처가 'model_summary.txt' 파일에 저장되었습니다.")

In [ ]:
import os
from torchsummary import summary
from contextlib import redirect_stdout

model = RNN()

# 새 폴더 생성 경로 설정
folder_path = '/content/drive/MyDrive/Colab Notebooks/graph/new_folder'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# 파일 저장 경로 설정
file_path = os.path.join(folder_path, 'model_summary.txt')

# 요약된 모델 아키텍처를 직접 파일에 리디렉션합니다.
with open(file_path, 'w') as f:
    with redirect_stdout(f):
        summary(model, (30, 5))

print(f"모델 아키텍처가 '{file_path}' 파일에 저장되었습니다.")


In [ ]:
short_term_label2 = []
for i in short_term_label:
if i >= 10:
    short_term_label2.append(3)

elif 0 <= i < 10:
    short_term_label2.append(2)

elif -10 < i < 0:
    short_term_label2.append(1)

elif i <= -10:
    short_term_label2.append(0)

In [ ]:
short_term_data[:]

In [ ]:
# term3_data
# 연속된 3개의 data를 묶어서 다시 구성.
term3_data = []
term3_label = []

label = data['Close'].to_numpy() # label은 마지막날종가와 다음날종가의 증가율로.

for i in range(len(std_data)-2):

    d = np.hstack([std_data[i,:], std_data[i+1,:], std_data[i+2,:]])
    l = (label[i+2]-label[i+1])/label[i+1]*100

    term3_data.append(d)
    term3_label.append(l)

term3_data = torch.tensor(term3_data).float()    # double형은 모델훈련시 가중치와 data형이 맞지 않으므로
term3_label = torch.tensor(term3_label).float()  # float형으로 변환한다.

print(term3_data.shape)
print(term3_label.shape)
print(term3_data[0])
print(term3_label[0])

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf

In [ ]:
apple = yf.Ticker("AAPL")
hist = apple.history(period="10y")
hist = hist.reset_index()
hist

In [ ]:
# hist[hist['Stock Splits'] != 0].index #주식분할에 의한 주가변동은 플랫폼에서 연속성을 유지하기위해 보정하기때문에 따로 전처리할 필요없음.

In [ ]:
hist.drop(['Date','Dividends','Stock Splits'], axis=1, inplace=True)
hist